In [108]:
import pulp
import pandas as pd

In [109]:
model_BESS = pulp.LpProblem('Minimize_cost', pulp.LpMinimize)

# Params

T = 24  # Time
E_charge = 93/100    # efficiency of BESS charging
E_discharge = 100/93    # efficiency of BESS charging
B = 750    # BESS capacity
W = 250    # Max power of BESS
P_base = 0    # Basefee


# Sets

PERIOD = list(range(0,T))
P_tou = [44.8, 44.8, 44.8, 44.8, 44.8, 44.8, 44.8, 44.8, 44.8, 59.2, 79.7, 79.7, 59.2, 79.7, 79.7, 79.7, 79.7, 59.2, 59.2, 59.2, 59.2, 59.2, 59.2, 44.8]
Load = pd.read_csv('load.csv', header=None)
PV = pd.read_csv('PV.csv', header=None)
L = Load[0]
S = PV[0]

# Variables

u = pulp.LpVariable.dicts('BESS State of Charge', [t for t in PERIOD], lowBound=0.2, upBound=0.8, cat='Continuous')
d_load = pulp.LpVariable.dicts('Diesel to Load at t', [t for t in PERIOD], lowBound=0, cat='Continuous')
d_bess = pulp.LpVariable.dicts('Diesel to BESS at t', [t for t in PERIOD], lowBound=0, cat='Continuous')
v_load = pulp.LpVariable.dicts('PV to Load at t', [t for t in PERIOD], lowBound=0, cat='Continuous')
v_bess = pulp.LpVariable.dicts('PV to BESS at t', [t for t in PERIOD], lowBound=0, cat='Continuous')
c = pulp.LpVariable.dicts('BESS to Load at t', [t for t in PERIOD], lowBound=0, cat='Continuous')

In [110]:
# Constraints

# Initial and Final SoC
model_BESS += u[0] == 0.5
model_BESS += u[T-1] == 0.5


# Load Constraint
for t in PERIOD:
    model_BESS += d_load[t] + v_load[t] + E_discharge * c[t] == L[t]


# PV Constraint
for t in PERIOD:
    model_BESS += v_bess[t] + v_load[t] == S[t]


# SoC Constraint
for t in range(0,T-2):
    model_BESS += u[t] + (E_charge * (d_bess[t] + v_bess[t]) - c[t]) / B == u[t+1]


# BESS power Constraint
for t in PERIOD:
    model_BESS += c[t] + E_charge * (d_bess[t] + v_bess[t]) <= W


# Objective Function
for t in PERIOD:
    model_BESS.objective += pulp.lpSum(P_tou[t] * (d_bess[t]+d_load[t]))


# Model Solve
model_BESS.solve(pulp.GUROBI())

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 96 rows, 144 columns and 304 nonzeros
Model fingerprint: 0xe4543ff9
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [4e+01, 8e+01]
  Bounds range     [2e-01, 8e-01]
  RHS range        [5e-01, 9e+03]
Presolve removed 52 rows and 78 columns
Presolve time: 0.00s
Presolved: 44 rows, 66 columns, 131 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1033697e+07   2.540000e+01   0.000000e+00      0s
      23    1.1348793e+07   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.134879345e+07
Gurobi status= 2


1